In [45]:

import numpy as np
import torch
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution3D, ZeroPadding3D, MaxPooling3D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

meta_data = './dataset/metadata/participant_d040722.csv'


In [46]:
metadata = pd.read_csv(meta_data)
metadata = metadata.query('rndgroup == 1 ')

#metadata
metadata = metadata[[ 'pid','lung_cancer', 'conflc', 'canc_rpt_link', 'cancyr', 'can_scr', 'last_screen_studyyr' ]]

def get_labels(pids, metadata_df):
    labels = metadata_df[metadata_df['pid'].isin(pids)]
    return pids, labels

cancerous = metadata.query('lung_cancer == 1 and canc_rpt_link == 1 and can_scr == 1 and cancyr == 0')
non_cancerous = metadata.query('lung_cancer == 0 and last_screen_studyyr == 0 ' )

test_positive = cancerous.iloc[170:]
train_positive = cancerous.iloc[0:170]
train_sample  = train_positive.sample(n = 110)

#return comma separated pids
def cs_pid(dataframe):

    comma_seperated_pids = ''
    for i, row in dataframe.iterrows():
        pid = row['pid'].astype(int)
        comma_seperated_pids += f' ,{pid}'
    print(comma_seperated_pids)





C:\Users\Woody\AppData\Local\Temp\ipykernel_512\2981791683.py:1: DtypeWarning: Columns (239,240,348) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(meta_data)


In [47]:
test_negetive = non_cancerous.iloc[600:]
train_negetive = non_cancerous.iloc[0:800]
train_sample  = train_negetive.sample(n = 200)

#return comma separated pids
cs_pid(train_sample)

 ,106001 ,108703 ,118725 ,124851 ,118192 ,202380 ,124952 ,130034 ,201264 ,116830 ,130824 ,116447 ,129973 ,103819 ,203427 ,126600 ,128343 ,126548 ,121706 ,206169 ,112313 ,209432 ,128052 ,210100 ,120354 ,203099 ,210698 ,102527 ,111877 ,210165 ,108789 ,128319 ,203563 ,117858 ,203333 ,114565 ,127682 ,206451 ,101746 ,112169 ,109454 ,102280 ,210964 ,103576 ,201852 ,130329 ,118021 ,206700 ,119929 ,109371 ,129179 ,109364 ,109324 ,105906 ,103146 ,115803 ,203491 ,111284 ,103151 ,121369 ,202982 ,103055 ,116625 ,102974 ,100766 ,128269 ,102023 ,122812 ,100568 ,110805 ,108754 ,120551 ,206672 ,108024 ,105764 ,117408 ,208527 ,132607 ,131536 ,103932 ,125826 ,204623 ,205620 ,209661 ,134164 ,100715 ,207666 ,109245 ,130193 ,121331 ,112766 ,203771 ,202733 ,128981 ,115411 ,125237 ,207325 ,120724 ,110681 ,133409 ,207877 ,126695 ,109510 ,201565 ,210745 ,209106 ,209979 ,206599 ,129633 ,118936 ,201139 ,128214 ,108900 ,130585 ,210210 ,126511 ,134526 ,111239 ,131970 ,209254 ,103764 ,127442 ,122663 ,202339 ,108820

In [49]:
import os 

def return_scan_path(path, sliceThreshold):
    scans_path = []
    for d in os.listdir(path):
        if os.path.isdir(f'{path}\{d}') :
            for dd in os.listdir(f'{path}\{d}'):
                if os.path.isdir(f'{path}\{d}\{dd}') :
                    for ddd in os.listdir(f'{path}\{d}\{dd}'):
                        if len(os.listdir(f'{path}\{d}\{dd}\{ddd}')) >= sliceThreshold: 
                            scans_path.append(f'{path}\{d}\{dd}\{ddd}')

    return scans_path
#return_scan_path(r'c:\Users\Woody\Desktop\lung cancer\dataset\train_positive\manifest-1681052108196\NLST', 100)

In [51]:
# data loader 
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

batch_size = 10
positives = np.array(os.listdir(f'c:/Users/Woody/Desktop/lung cancer/dataset/train_positive/manifest-1681052108196/segmented/'))
negetives = np.array(os.listdir(f'c:/Users/Woody/Desktop/lung cancer/dataset/train_negetive/manifest-1681053813358/segemented/'))
data = np.concatenate((positives, negetives))


def get_labels_from_fnames(names, metadata_df):
    pids = np.array([int(name[:-3]) for name in names ])
    labels = metadata_df[metadata_df['pid'].isin(pids)]

    reindexed_labels = labels.set_index('pid')
    reindexed_labels = reindexed_labels.reindex(pids)['lung_cancer'].values
    return pids, reindexed_labels

x, y = get_labels_from_fnames(data, metadata)

# test size should be changed 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.9, random_state=42)


def get_labels(pids, metadata_df):
    labels = metadata_df[metadata_df['pid'].isin(pids)]
    return pids, labels



class lungs_dataset(Dataset):
    def __init__(self, x, y , path ):
        self.x = x 
        self.y = y 
        self.path = path
        


    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        pid = self.x[idx]
        label = self.y[idx]

        fname = f'{self.path}{pid}.pt'
        tensor = torch.load(fname)

        return tensor , label

path_to_sdata = f'c:/Users/Woody/Desktop/lung cancer/dataset/segmented_lungs/' 
training_data = lungs_dataset(X_train, y_train, path_to_sdata)
#tloader = training data loader 
tloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
iter_tloader = iter(tloader)



In [52]:
import torch.nn as nn 
import torch
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # 3D convolutional layers
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=32, kernel_size=3)
        #self.conv2 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3)
        #self.conv3 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3)
        
        # Max pooling layer
        self.pool = nn.MaxPool3d(kernel_size=2)
        
        # Fully connected layers
        #self.fc1 = nn.Linear(in_features=128 * 6 * 6 * 6, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=10)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.2)
    
    def forward(self, x):
        # Pass input through 3D convolutional layers and max pooling layer
        x = self.pool(torch.relu(self.conv1(x)))
        #x = self.pool(torch.relu(self.conv2(x)))
        #x = self.pool(torch.relu(self.conv3(x)))
        
        # Flatten the output from 3D convolutional layers
        x = x.view(-1, 128 * 6 * 6 * 6)
        
        # Pass flattened output through fully connected layers
        #x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        
        return x
    

net = Net()
data, _ = iter_tloader.next()
data.shape

torch.Size([10, 320, 410, 410])

In [53]:
data = data.unsqueeze(1).short()
net(data)
#summary( net , (1 ,320, 410, 410) )
print(net)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:73] data. DefaultCPUAllocator: not enough memory: you tried to allocate 33878753280 bytes. Buy new RAM!

In [20]:
epochs = 10
train_losses, test_losses, accuracy  = [] , [] , []
learning_rate = 0.001


criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr= learning_rate)



In [ ]:
total = 0 
correct = 0
for epoch in range(epochs):
    running_loss = 0
    for i, data in enumerate(tloader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        
        print(outputs)
        print(labels)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        train_losses.append(loss.item())
        with torch.no_grad():
            patch_correct = np.count_nonzero(np.round(outputs) == labels)
            correct += patch_correct
            total += batch_size
            accuracy.append(correct/ total)
            print(f'patch_correct is : {patch_correct} \n output is {np.round(outputs) } \n labes are { labels} ')
            


        
        if i % 15 == 14:
            print(f'running loss for epoch {epoch } is {running_loss / 15}')
            running_loss = 0
        

In [1]:
#fix the dataset and model and try to run on a few scans. 
# 1. this includes running the model on GPUs 
# 2. break the lungs into smaller parts to be able to run it (first try train the model with full ct-scans)
#clean the code, put everything into relevant functions and classes, break it into 1. preprocessing 2. dataloading 3. model and training  


